#  Tradução de Textos - Implantação
## Utilização do modelo [MarianMT](https://huggingface.co/transformers/model_doc/marian.html) para tradução. 


* Neste exempo a tradução é feito do inglês para o português, mas ela pode ser feita em qualquer uma das línguas suportadas pelo MarianMT. 
* Para adaptar para traduções em outras línguas é necessário verificacar se há o modelo pré treinado disponível no MarianMT e adaptar o truncamento de strings do [spacy](https://spacy.io/usage/models) para o idioma desejado
* A métrica computada é o [sacrebleu](https://https://github.com/mjpost/sacrebleu) 


Preencha aqui com detalhes sobre a tarefa.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/nlp-marianmt-translator/marianmt_model.py

--2020-10-27 14:49:38--  https://raw.githubusercontent.com/platiagro/tasks/main/tasks/nlp-marianmt-translator/marianmt_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.92.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.92.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6628 (6.5K) [text/plain]
Saving to: ‘marianmt_model.py’

marianmt_model.py   100%[===================>]   6.47K  --.-KB/s    in 0.004s  

2020-10-27 14:49:38 (1.70 MB/s) - ‘marianmt_model.py’ saved [6628/6628]



In [2]:
%%writefile model.py
import joblib
import pandas as pd
import numpy as np
from marianmt_model import MarianMTTranslator


class Model:
    def __init__(self):
        artifacts = joblib.load("/tmp/data/ocr.joblib")
        self.hyperparams = artifacts["hyperparams"]
        self.model_params = artifacts["model_params"]
        self.columns = artifacts["columns"]
    
    def predict(self, X, feature_names, meta=None):

        model = MarianMTTranslator(self.hyperparams,self.model_params)
        y = model.predict(X)
        return y

Writing model.py


## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [5]:
# %%writefile contract.json
# {
#     "features": [
#         {
#             "name": "text",
#             "ftype": "categorical",
#             "values": ["There are many levels of perversity at play. And yet it is transfixing."]
#         },
#     ],
#     "targets": []
# }

Overwriting contract.json


In [7]:
# from platiagro.deployment import test_deployment

# test_deployment("contract.json")